In [2]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd


def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito") #opens scraper in incognito mode
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs
            
            
        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [17]:

path = "/Users/flationschool/Course_Materials/nyc-mhtn-ds-010620-lectures/Final_Project/chromedriver"

df = get_jobs('data scientist',100, False, path, 15)

#df.to_csv('ds_glassdoor_jobs.csv', index = False)

 x out worked
Progress: 0/100
Progress: 1/100
Progress: 2/100
Progress: 3/100
Progress: 4/100
Progress: 5/100
Progress: 6/100
Progress: 7/100
Progress: 8/100
Progress: 9/100
Progress: 10/100
Progress: 11/100
Progress: 12/100
Progress: 13/100
Progress: 14/100
Progress: 15/100
Progress: 16/100
Progress: 17/100
Progress: 18/100
Progress: 19/100
Progress: 20/100
Progress: 21/100
Progress: 22/100
Progress: 23/100
Progress: 24/100
Progress: 25/100
Progress: 26/100
Progress: 27/100
Progress: 28/100
Progress: 29/100
 x out failed
Progress: 30/100
Progress: 31/100
Progress: 32/100
Progress: 33/100
Progress: 34/100
Progress: 35/100
Progress: 36/100
Progress: 37/100
Progress: 38/100
Progress: 39/100
Progress: 40/100
Progress: 41/100
Progress: 42/100
Progress: 43/100
Progress: 44/100
Progress: 45/100
Progress: 46/100
Progress: 47/100
Progress: 48/100
Progress: 49/100
Progress: 50/100
Progress: 51/100
Progress: 52/100
Progress: 53/100
Progress: 54/100
Progress: 55/100
Progress: 56/100
Progress: 57/

In [16]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$108K-$145K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nSumma...",4.1,ManTech\n4.1,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
1,Data Scientist,$108K-$145K (Glassdoor est.),Who We Are\n\nWere Americas largest mortgage l...,3.9,Quicken Loans\n3.9,"Detroit, MI","Detroit, MI",10000+ employees,1985,Company - Private,Lending,Finance,$10+ billion (USD),"Citi, Bank of America, Wells Fargo"
2,Data Scientist,$108K-$145K (Glassdoor est.),GEICO’s Data Science team uses predictive anal...,3.3,GEICO\n3.3,"Chevy Chase, MD","Washington, DC",10000+ employees,1936,Subsidiary or Business Segment,Insurance Carriers,Insurance,$10+ billion (USD),-1
3,Data Scientist,$108K-$145K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310781\n\n...,3.8,Pacific Northwest National Laboratory\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,Data Scientist,$108K-$145K (Glassdoor est.),JOB CATEGORY:\n\nInformation Services\n\nREQUI...,3.9,Mars\n3.9,New York State,"Mc Lean, VA",10000+ employees,1911,Company - Private,Food & Beverage Manufacturing,Manufacturing,$10+ billion (USD),-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Data Scientist Buildings Research,$49K-$85K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310585\n\n...,3.8,Pacific Northwest National Laboratory\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
96,Senior Research Scientist,$49K-$85K (Glassdoor est.),The Senior Research Scientist will support the...,3.5,LifeVantage\n3.5,"Sandy, UT","Sandy, UT",201 to 500 employees,2009,Company - Public,"Health, Beauty, & Fitness",Consumer Services,$100 to $500 million (USD),-1
97,Data Scientist,$49K-$85K (Glassdoor est.),Roles and responsibilities include the followi...,2.3,Vanda Pharmaceuticals\n2.3,"Washington, DC","Washington, DC",201 to 500 employees,2003,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$100 to $500 million (USD),-1
98,Lead Data Scientist,$49K-$85K (Glassdoor est.),Position: Lead Data Scientist\n\nLocation: Cin...,3.8,PatientPoint\n3.8,"Cincinnati, OH","Cincinnati, OH",201 to 500 employees,1987,Company - Private,Advertising & Marketing,Business Services,$100 to $500 million (USD),"Outcome Health, Health Media Network, Mesmeriz..."


In [25]:
apple = pd.concat([df, ef], ignore_index = True)
apple

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$108K-$145K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nSumma...",4.1,ManTech\n4.1,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
1,Data Scientist,$108K-$145K (Glassdoor est.),Who We Are\n\nWere Americas largest mortgage l...,3.9,Quicken Loans\n3.9,"Detroit, MI","Detroit, MI",10000+ employees,1985,Company - Private,Lending,Finance,$10+ billion (USD),"Citi, Bank of America, Wells Fargo"
2,Data Scientist,$108K-$145K (Glassdoor est.),GEICO’s Data Science team uses predictive anal...,3.3,GEICO\n3.3,"Chevy Chase, MD","Washington, DC",10000+ employees,1936,Subsidiary or Business Segment,Insurance Carriers,Insurance,$10+ billion (USD),-1
3,Data Scientist,$108K-$145K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310781\n\n...,3.8,Pacific Northwest National Laboratory\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,Data Scientist,$108K-$145K (Glassdoor est.),JOB CATEGORY:\n\nInformation Services\n\nREQUI...,3.9,Mars\n3.9,New York State,"Mc Lean, VA",10000+ employees,1911,Company - Private,Food & Beverage Manufacturing,Manufacturing,$10+ billion (USD),-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Data Scientist Buildings Research,$49K-$85K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310585\n\n...,3.8,Pacific Northwest National Laboratory\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
196,Senior Research Scientist,$49K-$85K (Glassdoor est.),The Senior Research Scientist will support the...,3.5,LifeVantage\n3.5,"Sandy, UT","Sandy, UT",201 to 500 employees,2009,Company - Public,"Health, Beauty, & Fitness",Consumer Services,$100 to $500 million (USD),-1
197,Lead Data Scientist,$49K-$85K (Glassdoor est.),Position: Lead Data Scientist\n\nLocation: Cin...,3.8,PatientPoint\n3.8,"Cincinnati, OH","Cincinnati, OH",201 to 500 employees,1987,Company - Private,Advertising & Marketing,Business Services,$100 to $500 million (USD),"Outcome Health, Health Media Network, Mesmeriz..."
198,Data Scientist,$49K-$85K (Glassdoor est.),We are looking for an enthusiastic Data Scient...,4.3,Credera\n4.3,"Dallas, TX","Dallas, TX",201 to 500 employees,1999,Subsidiary or Business Segment,Consulting,Business Services,Unknown / Non-Applicable,-1


In [26]:
apple.duplicated().sum()

93

In [29]:
apple.drop_duplicates(inplace = True)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
164,Data Engineer,$60K-$100K (Glassdoor est.),We're looking for 2 developers. The candidates...,3.1,Concord USA\n3.1,"Hopkins, MN","Hopkins, MN",201 to 500 employees,2003,Company - Private,Consulting,Business Services,$50 to $100 million (USD),-1
171,Staff Scientist,$60K-$100K (Glassdoor est.),JOB TITLE: Staff Scientist\n\nLOCATION: Geisin...,3.5,Geisinger\n3.5,"Danville, PA","Danville, PA",10000+ employees,1915,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$500 million to $1 billion (USD),"UPMC, Community Health Systems, Mayo Clinic"
193,Data Scientist,$60K-$100K (Glassdoor est.),Roles and responsibilities include the followi...,2.3,Vanda Pharmaceuticals\n2.3,"Washington, DC","Washington, DC",201 to 500 employees,2003,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$100 to $500 million (USD),-1
198,Data Scientist,$49K-$85K (Glassdoor est.),We are looking for an enthusiastic Data Scient...,4.3,Credera\n4.3,"Dallas, TX","Dallas, TX",201 to 500 employees,1999,Subsidiary or Business Segment,Consulting,Business Services,Unknown / Non-Applicable,-1
199,Data Analytics Engineer,$49K-$85K (Glassdoor est.),Our Data Analytics Engineer will engineer eleg...,3.9,QOMPLX\n3.9,"Reston, VA","Reston, VA",51 to 200 employees,2015,Company - Private,Enterprise Software & Network Solutions,Information Technology,$10 to $25 million (USD),-1


In [34]:
apple2

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"See Tickets, TicketWeb, Vendini"
952,952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,-1
953,953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),-1
954,954,Data Engineer,-1,Loading...\n\nTitle: Data Engineer\n\nLocation...,4.8,IGNW\n4.8,"Austin, TX","Portland, OR",201 to 500 employees,2015,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),Slalom


In [37]:
apple2.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [55]:
finaldf1 = pd.concat([apple, apple2], ignore_index = True)

In [61]:
finaldf1.drop_duplicates(inplace = True)

In [62]:
finaldf1

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$108K-$145K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nSumma...",4.1,ManTech\n4.1,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
1,Data Scientist,$108K-$145K (Glassdoor est.),Who We Are\n\nWere Americas largest mortgage l...,3.9,Quicken Loans\n3.9,"Detroit, MI","Detroit, MI",10000+ employees,1985,Company - Private,Lending,Finance,$10+ billion (USD),"Citi, Bank of America, Wells Fargo"
2,Data Scientist,$108K-$145K (Glassdoor est.),GEICO’s Data Science team uses predictive anal...,3.3,GEICO\n3.3,"Chevy Chase, MD","Washington, DC",10000+ employees,1936,Subsidiary or Business Segment,Insurance Carriers,Insurance,$10+ billion (USD),-1
3,Data Scientist,$108K-$145K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310781\n\n...,3.8,Pacific Northwest National Laboratory\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,Data Scientist,$108K-$145K (Glassdoor est.),JOB CATEGORY:\n\nInformation Services\n\nREQUI...,3.9,Mars\n3.9,New York State,"Mc Lean, VA",10000+ employees,1911,Company - Private,Food & Beverage Manufacturing,Manufacturing,$10+ billion (USD),-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,Data Scientist,-1,Steel Point Solutions is an amazing SBA Certif...,4.7,"Steel Point Solutions, LLC\n4.7","Alexandria, VA","Calverton, MD",1 to 50 employees,2013,Other Organization,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1
1032,Data Scientists,-1,Position Description:\nThe Analytics Research ...,3.3,Miracle Software Systems\n3.3,Michigan,"Novi, MI",51 to 200 employees,1994,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),"CSC, IBM"
1035,"Associate Scientist / Sr. Associate Scientist,...",$59K-$125K (Glassdoor est.),"Who we are\n\n23andMe, the leading consumer ge...",4,23andMe\n4.0,"South San Francisco, CA","Sunnyvale, CA",501 to 1000 employees,2006,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Unknown / Non-Applicable,"Ancestry, Verily Life Sciences, Abbott Laborat..."
1052,Machine Learning Engineer (NLP),$80K-$142K (Glassdoor est.),CK-12’s mission is to provide free access to o...,4.1,CK-12 Foundation\n4.1,"Palo Alto, CA","Palo Alto, CA",1 to 50 employees,2007,Company - Private,K-12 Education,Education,Unknown / Non-Applicable,-1


In [65]:
finaldf2 = pd.read_csv('glassdoor_jobs2.csv')

In [67]:
lastdf = pd.concat([finaldf1, finaldf2], ignore_index = True)

In [69]:
lastdf.duplicated().sum()

1

In [70]:
lastdf.drop_duplicates(inplace = True)

In [71]:
lastdf.to_csv('ds_glassdoor_jobs.csv', index = False)

(1252, 14)